<a href="https://colab.research.google.com/github/mbouaida/ImmoAI/blob/master/BertForClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm

In [ ]:
torch.__version__

'2.6.0+cu124'

In [ ]:
!rm -rf /root/.cache/huggingface/datasets
!pip install --upgrade --quiet datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
dataset = load_dataset("ag_news", cache_dir="/content/hf_cache")
train_ds = dataset["train"]
test_ds = dataset["test"]
labels = train_ds.features["label"].names
print(f"{len(train_ds)} exemples de train, {len(test_ds)} de test. Classes = {labels}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

120000 exemples de train, 7600 de test. Classes = ['World', 'Sports', 'Business', 'Sci/Tech']


In [ ]:
# Prendre le premier exemple
example = train_ds[0]
text = example["text"]
label_id = example["label"]
label_name = labels[label_id]

print("🔹 Texte brut :")
print(text)
print("\n🔹 Classe :", label_name)

# Charger le tokenizer BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization
encoded = tokenizer(text)

print("\n🔹 Tokens :")
print(encoded.tokens())

print("\n🔹 IDs :")
print(encoded["input_ids"])

print("\n🔹 Reconversion IDs -> Tokens :")
print(tokenizer.convert_ids_to_tokens(encoded["input_ids"]))

🔹 Texte brut :
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

🔹 Classe : Business


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


🔹 Tokens :
['[CLS]', 'wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short', '-', 'sellers', ',', 'wall', 'street', "'", 's', 'd', '##wind', '##ling', '\\', 'band', 'of', 'ultra', '-', 'cy', '##nic', '##s', ',', 'are', 'seeing', 'green', 'again', '.', '[SEP]']

🔹 IDs :
[101, 2813, 2358, 1012, 6468, 15020, 2067, 2046, 1996, 2304, 1006, 26665, 1007, 26665, 1011, 2460, 1011, 19041, 1010, 2813, 2395, 1005, 1055, 1040, 11101, 2989, 1032, 2316, 1997, 11087, 1011, 22330, 8713, 2015, 1010, 2024, 3773, 2665, 2153, 1012, 102]

🔹 Reconversion IDs -> Tokens :
['[CLS]', 'wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short', '-', 'sellers', ',', 'wall', 'street', "'", 's', 'd', '##wind', '##ling', '\\', 'band', 'of', 'ultra', '-', 'cy', '##nic', '##s', ',', 'are', 'seeing', 'green', 'again', '.', '[SEP]']


In [ ]:
# 1️⃣ Charger le tokenizer BERT pré-entraîné (version uncased = tout en minuscules)
# Ce tokenizer convertira du texte brut → tokens BERT → IDs (entiers)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 🔍 Exemple pour visualiser le comportement du tokenizer
example_text = "Wall St. Bears Claw Back Into the Black (Reuters)"
encoded_example = tokenizer(example_text, padding="max_length", truncation=True, max_length=20)

print("Texte brut :", example_text)
print("Tokens :", tokenizer.tokenize(example_text))
print("Input IDs :", encoded_example['input_ids'])
print("Tokens depuis IDs :", tokenizer.convert_ids_to_tokens(encoded_example['input_ids']))
print("Attention mask :", encoded_example['attention_mask'])


Texte brut : Wall St. Bears Claw Back Into the Black (Reuters)
Tokens : ['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')']
Input IDs : [101, 2813, 2358, 1012, 6468, 15020, 2067, 2046, 1996, 2304, 1006, 26665, 1007, 102, 0, 0, 0, 0, 0, 0]
Tokens depuis IDs : ['[CLS]', 'wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]


In [ ]:
# 2️⃣ Définir une fonction de prétraitement pour appliquer le tokenizer à un batch d'exemples

def preprocess(batch):
    """
    Applique le tokenizer sur un batch de textes.
    - padding="max_length" : complète à une longueur fixe avec des [PAD]
    - truncation=True : coupe les textes trop longs à max_length
    - max_length=128 : longueur fixe des séquences
    """
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)


In [ ]:
# 4️⃣ Appliquer la fonction de prétraitement à tout le dataset (train + test)
# - batched=True : on applique la fonction sur un batch (liste de textes), pas un seul à la fois → plus rapide
# - remove_columns(["text"]) : on supprime la colonne brute "text" après tokenisation
train_ds = train_ds.map(preprocess, batched=True).remove_columns(["text"])
test_ds  = test_ds.map(preprocess, batched=True).remove_columns(["text"])

# ➕ Exemple de résultat
print("Exemple tokenisé dans le train_ds :")
print(train_ds[0])


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Exemple tokenisé dans le train_ds :
{'label': 2, 'input_ids': [101, 2813, 2358, 1012, 6468, 15020, 2067, 2046, 1996, 2304, 1006, 26665, 1007, 26665, 1011, 2460, 1011, 19041, 1010, 2813, 2395, 1005, 1055, 1040, 11101, 2989, 1032, 2316, 1997, 11087, 1011, 22330, 8713, 2015, 1010, 2024, 3773, 2665, 2153, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1

In [ ]:
# 5️⃣ On définit le format final des datasets :
# - "torch" : les colonnes seront automatiquement converties en tenseurs PyTorch
# - columns=... : on garde uniquement les colonnes nécessaires au modèle
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# ➕ Vérification : chaque entrée est maintenant prête à être utilisée par un modèle PyTorch
print("Exemple final prêt pour modèle PyTorch :")
print(train_ds[0])

Exemple final prêt pour modèle PyTorch :
{'label': tensor(2), 'input_ids': tensor([  101,  2813,  2358,  1012,  6468, 15020,  2067,  2046,  1996,  2304,
         1006, 26665,  1007, 26665,  1011,  2460,  1011, 19041,  1010,  2813,
         2395,  1005,  1055,  1040, 11101,  2989,  1032,  2316,  1997, 11087,
         1011, 22330,  8713,  2015,  1010,  2024,  3773,  2665,  2153,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,    

In [ ]:
from torch.utils.data import DataLoader

# 1️⃣ Création du DataLoader pour les données d'entraînement
train_loader = DataLoader(
    dataset=train_ds,   # Le dataset PyTorch-ready (avec des tensores)
    batch_size=16,      # Chaque batch contiendra 16 exemples (input_ids, attention_mask, label)
    shuffle=True        # On mélange les données à chaque époque (très important pour l'entraînement)
)

# 2️⃣ Création du DataLoader pour les données de test
test_loader = DataLoader(
    dataset=test_ds,    # Le dataset de test
    batch_size=64,      # On peut prendre des batches plus grands pour le test (pas de rétropropagation)
    shuffle=False       # Pas besoin de mélanger les données pour l'évaluation
)


In [ ]:
# Obtenir un batch depuis le DataLoader
batch = next(iter(train_loader))

print("🔸 Clés du batch :", batch.keys())  # ['input_ids', 'attention_mask', 'label']

print("\n🔹 input_ids (forme):", batch['input_ids'].shape)
print(batch['input_ids'][0])  # Affiche les IDs du premier exemple du batch

print("\n🔹 attention_mask (forme):", batch['attention_mask'].shape)
print(batch['attention_mask'][0])  # Affiche le masque correspondant

print("\n🔹 label :", batch['label'])  # Les labels pour chaque exemple du batch

🔸 Clés du batch : dict_keys(['label', 'input_ids', 'attention_mask'])

🔹 input_ids (forme): torch.Size([16, 128])
tensor([  101,  2317,  2160,  1024,  7016,  5894,  2442,  2022,  2992,  1996,
         5747,  3447,  2623,  9317,  2008,  2009,  2097,  2448,  2041,  1997,
        17519,  2075,  2282,  2000,  6133,  1996,  2231,  1001,  4464,  1025,
         1055,  5294, 23733,  3791,  1999,  2048,  3134,  1010,  5128,  2062,
         3778,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,

In [ ]:
# 1️⃣ Détection automatique de l'appareil (GPU si disponible, sinon CPU)
# torch.cuda.is_available() retourne True si un GPU CUDA est disponible (comme dans Google Colab)
# torch.device permet ensuite de le stocker dans une variable utilisable partout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2️⃣ Chargement d’un modèle BERT pré-entraîné pour une tâche de classification de texte
# - "bert-base-uncased" : modèle BERT avec texte en minuscules uniquement
# - AutoModelForSequenceClassification : ajoute une couche de classification à la sortie de BERT
# - num_labels = nombre de classes de sortie (par exemple 4 pour AG News)
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(labels)  # labels = ["World", "Sports", "Business", "Sci/Tech"]
)

# 3️⃣ Déplacement du modèle sur l'appareil choisi (GPU si disponible, sinon CPU)
# C'est obligatoire pour que les données et le modèle soient sur le même device
model.to(device)

# 4️⃣ Définition de l’optimiseur utilisé pour ajuster les poids du modèle
# - AdamW : une version de l'algorithme Adam avec régularisation de poids (weight decay), recommandé pour BERT
# - model.parameters() : tous les paramètres entraînables du modèle
# - lr = learning rate = taux d’apprentissage (valeur typique pour BERT = 2e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from tqdm.auto import tqdm
from colorama import Fore, Style

# ⚙️ Réduire le dataset pour un entraînement rapide (ex: 200 exemples)
small_train_ds = train_ds.select(range(200))
small_train_loader = DataLoader(small_train_ds, batch_size=16, shuffle=True)

# 🔧 Mettre le modèle en mode entraînement (active dropout, etc.)
model.train()

# 📦 Entraînement sur 3 époques
for epoch in range(3):
    print(Fore.CYAN + f"\n🔁 DÉBUT DE L'ÉPOQUE {epoch + 1}" + Style.RESET_ALL)

    # tqdm = barre de progression
    loop = tqdm(small_train_loader, leave=True)

    for step, batch in enumerate(loop):
        # 📲 Déplacement de chaque élément du batch vers le bon device (GPU ou CPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # ✅ Corriger la clé : label → labels (ce que le modèle attend)
        batch["labels"] = batch.pop("label")

        # 1️⃣ Forward pass → le modèle prédit les logits + calcule la loss automatiquement
        outputs = model(**batch)
        loss = outputs.loss  # la perte entre prédiction et vraie classe

        # 2️⃣ Remise à zéro du gradient accumulé
        optimizer.zero_grad()

        # 3️⃣ Backward pass → calcul des gradients
        loss.backward()

        # 4️⃣ Optimizer step → mise à jour des poids du modèle
        optimizer.step()

        # 🖨️ Affichage de l'évolution en live
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=float(loss.item()))

        if step % 5 == 0:
            print(Fore.YELLOW + f"   ➤ Step {step}: Loss = {loss.item():.4f}" + Style.RESET_ALL)

    print(Fore.GREEN + f"✅ Fin de l'époque {epoch + 1}" + Style.RESET_ALL)



🔁 DÉBUT DE L'ÉPOQUE 1


  0%|          | 0/13 [00:00<?, ?it/s]

   ➤ Step 0: Loss = 1.3795
   ➤ Step 5: Loss = 1.0574
   ➤ Step 10: Loss = 0.7950
✅ Fin de l'époque 1

🔁 DÉBUT DE L'ÉPOQUE 2


  0%|          | 0/13 [00:00<?, ?it/s]

   ➤ Step 0: Loss = 0.6786
   ➤ Step 5: Loss = 0.5967
   ➤ Step 10: Loss = 0.4956
✅ Fin de l'époque 2

🔁 DÉBUT DE L'ÉPOQUE 3


  0%|          | 0/13 [00:00<?, ?it/s]

   ➤ Step 0: Loss = 0.5832
   ➤ Step 5: Loss = 0.3106
   ➤ Step 10: Loss = 0.3860
✅ Fin de l'époque 3


In [ ]:
%pip install colorama